In [4]:
import numpy as np
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import scipy as scio
from scipy.spatial.distance import pdist
from scipy.linalg import cho_factor, cho_solve, cholesky
from sklearn.metrics.pairwise import rbf_kernel, pairwise_kernels
from sklearn.metrics import mean_squared_error
from sklearn.utils import check_array, check_random_state
from sklearn.linear_model.ridge import _solve_cholesky_kernel as kernel_solve
from time import time
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
import fbpca

%matplotlib inline
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

## Resources

#### Original Formulation



## Generate Data

In [5]:
# Import dataset
from scipy.io import loadmat
import requests

data_url = 'https://github.com/mli/nystrom/raw/master/satimage.mat'
r = requests.get(data_url, allow_redirects=True)
open('satire.mat', 'wb').write(r.content)

data = scio.io.loadmat('satire.mat')['D'].toarray()
print('Size of data: {}'.format(data.shape))

n_samples = data.shape[0]
random_state = 123

Size of data: (4435, 36)


## Nystrom Approximation of a Kernel Matrix

#### Kernel Matrix of Data

In [6]:
sigma = np.mean(pdist(data, metric='euclidean'))
gamma = 1 / (2 * sigma**2)
n_jobs = 1
kernel = 'rbf'

K = pairwise_kernels(data, metric=kernel, n_jobs=n_jobs, gamma=gamma)
print('Size of Kernel matrix: ', K.shape)

Size of Kernel matrix:  (4435, 4435)


In [7]:
%timeit pairwise_kernels(data, metric=kernel, n_jobs=1, gamma=gamma)
%timeit rbf_kernel(data, gamma=gamma)

np.testing.assert_array_equal(pairwise_kernels(data, metric=kernel, n_jobs=1, gamma=gamma),
                              rbf_kernel(data, gamma=gamma))

398 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
390 ms ± 21.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Sampling

In [8]:
# uniform sampling without replacement

generator = check_random_state(random_state)
random_indices = generator.permutation(n_samples)

# choose the number of column indices
n_column_indices = 200

# choose 200 samples
column_indices = random_indices[:n_column_indices]

# choose the columns randomly from the matrix
C = K[:, column_indices]

print('Size of the sampled K matrix, C: ', C.shape)

# get the other sampled columns
W = C[column_indices, :]

print('Size of m-by-m intersection matrix, W: ', W.shape)

Size of the sampled K matrix, C:  (4435, 200)
Size of m-by-m intersection matrix, W:  (200, 200)


#### SVD

In [13]:
k_components = 100
t0 = time()
# Perform the normal SVD
print('Perform Normal SVD...')
U, D, V = np.linalg.svd(W, full_matrices=False)

U = U[:, :k_components]
V = V[:, :k_components]
D = D[:k_components]

U_approx = np.sqrt(n_column_indices / n_samples) * C.dot(U).dot(np.diag(D**(-1)))
D_approx = (n_samples / n_column_indices) * np.diag(D)
K_approx = U_approx.dot(D_approx).dot(U_approx.T)

print('Shape of U_approx: {}'.format(U_approx.shape))
print('Shape of D_approx: {}'.format(D_approx.shape))

W_approx = U.dot(np.diag(D)).dot(U.T)
K_approx2 = C.dot(np.linalg.pinv(W_approx)).dot(C.T)

err = np.linalg.norm(K - K_approx, 'fro')
err_2 = np.linalg.norm(K - K_approx2, 'fro')
print('Error (Normal): {:.3e}'.format(err))
print('Error (FBPCA) CWC.T: {:.3e}'.format(err_2))


Perform Normal SVD...
Shape of U_approx: (4435, 100)
Shape of D_approx: (100, 100)
Error (Normal): 4.915e+00
Error (FBPCA) CWC.T: 4.915e+00


#### Randomized SVD (FB)

In [24]:
print('Perform FB RSVD...')
U_frand, D_frand, V_frand = fbpca.pca(W, k=k_components, raw=True)

U_fapprox = np.sqrt(n_column_indices / n_samples) * C.dot(U_frand).dot(np.diag(D_frand**(-1)))
D_fapprox = (n_samples / n_column_indices) * np.diag(D_frand)
K_fapprox = U_fapprox.dot(D_fapprox).dot(U_fapprox.T)

W_fapprox = U_frand.dot(np.diag(D_frand)).dot(U_frand.T)
K_fapprox2 = C.dot(np.linalg.pinv(W_fapprox)).dot(C.T)

err_fb = np.linalg.norm(K - K_fapprox, 'fro')
err_fb2 = np.linalg.norm(K - K_fapprox2, 'fro')
print('Error (FBPCA): {:.3e}'.format(err_fb))
print('Error (FBPCA) CWC.T: {:.3e}'.format(err_fb2))


Perform FB RSVD...
Error (FBPCA): 4.922e+00
Error (FBPCA) CWC.T: 4.922e+00


#### Randomized SVD (Scikit)

In [25]:
print('Perform scikit RSVD...')
U_rand, D_rand, V_rand = randomized_svd(W, n_components=k_components)

U_rapprox = np.sqrt(n_column_indices / n_samples) * C.dot(U_rand).dot(np.diag(D_rand**(-1)))
D_rapprox = (n_samples / n_column_indices) * np.diag(D_rand)
K_rapprox = U_rapprox.dot(D_rapprox).dot(U_rapprox.T)

W_rapprox = U_rand.dot(np.diag(D_rand)).dot(U_rand.T)
K_rapprox2 = C.dot(np.linalg.pinv(W_rapprox)).dot(C.T)

err_r = np.linalg.norm(K - K_fapprox, 'fro')
err_r2 = np.linalg.norm(K - K_rapprox2, 'fro')

print('Error (FBPCA): {:.3e}'.format(err_r))
print('Error (FBPCA) CWC.T: {:.3e}'.format(err_r2))


Perform scikit RSVD...
Error (FBPCA): 4.922e+00
Error (FBPCA) CWC.T: 4.915e+00


In [26]:
print('Time Experiment for normal SVD.')
%timeit np.linalg.svd(W, full_matrices=False);

print('\nTime Experiment for randomized SVD (fb).')
%timeit fbpca.pca(W, k=k_components, raw=True, n_iter=3);

print('\nTime Experiment for randomized SVD (scikit).')
%timeit randomized_svd(W, n_components=k_components);

Time Experiment for normal SVD.
27.1 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Time Experiment for randomized SVD (fb).
55.7 ms ± 4.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Time Experiment for randomized SVD (scikit).
45.4 ms ± 946 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Data Approximation

#### Scikit-Learn Implementation

##### Sampling

In [27]:
# Nystrom approximation of K
eps = 1e-12

# size of the data
n_samples = data.shape[0]

# choose the number of components
k_components = 100
k_components = min(n_samples, k_components)

# perform rnadom uniform sampling without replacement
indices = np.random.permutation(n_samples)
basis_indices = indices[:k_components]
basis = data[basis_indices]

print('Size of indices: {}'.format(indices.shape))
print('Size of basis indices: {}'.format(basis_indices.shape))
print('Size of basis: {}'.format(basis.shape))

Size of indices: (4435,)
Size of basis indices: (100,)
Size of basis: (100, 36)


##### Construct Kernel Matrices

In [32]:
# construct the basis kernel
basis_gamma = 1 / (2 * np.mean(pdist(basis, metric='euclidean')))
basis_kernel = pairwise_kernels(basis, metric=kernel, n_jobs=n_jobs, gamma=gamma)

##### SVD - Get the basis vectors

In [33]:
# get the basis vectors

# Perform the normal SVD
print('Perform Normal SVD...')
U, D, V = np.linalg.svd(basis_kernel, full_matrices=False)

# use toleraance for eigenvalues
S = np.maximum(D, eps)

Perform Normal SVD...


In [34]:
# Get normalization
normalization = np.dot(U / np.sqrt(S), V)

# get components and indices
components = basis
component_indices = basis_indices

print('Size of normalization: {}'.format(normalization.shape))
print('Size of components: {}'.format(components.shape))
print('Size of component indices: {}'.format(component_indices.shape))

Size of normalization: (100, 100)
Size of components: (100, 36)
Size of component indices: (100,)


##### Randomized SVD

In [38]:
print('Perform scikit RSVD...')
U_rand, D_rand, V_rand = randomized_svd(basis_kernel, n_components=k_components)

# use toleraance for eigenvalues
S_rand = np.maximum(D, eps)

normalization_r = np.dot(U_rand / np.sqrt(S_rand), V_rand)

print('Size of normalization: {}'.format(normalization_r.shape))
print('Size of components: {}'.format(components.shape))
print('Size of component indices: {}'.format(component_indices.shape))

Perform scikit RSVD...
Size of normalization: (100, 100)
Size of components: (100, 36)
Size of component indices: (100,)


#### Transform New Data